In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [20]:
# Configure Brave browser
brave_path = "/usr/bin/brave-browser"
chromedriver_path = "/home/mehedihassan/Documents/Mastercourse/chromedriver"

options = Options()
options.binary_location = brave_path
options.add_argument("--start-maximized")  # Remove to run headless

service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=service, options=options)

url = "https://en.wikipedia.org/wiki/List_of_countries_by_renewable_electricity_production"
driver.get(url)

In [62]:
headers = [ header.text.replace('.','') for header in driver.find_elements(By.CLASS_NAME, "headerSort")]
headers

['Location',
 'Renew',
 'Hydro',
 'Wind',
 'Solar',
 'Bio',
 'Geo',
 'Year',
 'Location',
 'Renew',
 'Hydro',
 'Wind',
 'Solar',
 'Bio',
 'Geo',
 'Year']

In [63]:
twh_header = headers[:8]
percent_header = headers[8:len(headers)]

In [65]:
print(twh_header, percent_header)

['Location', 'Renew', 'Hydro', 'Wind', 'Solar', 'Bio', 'Geo', 'Year'] ['Location', 'Renew', 'Hydro', 'Wind', 'Solar', 'Bio', 'Geo', 'Year']


In [77]:
raw_data = [ data.text.split('\n') for data in driver.find_elements(By.CLASS_NAME, "wikitable")]
len(raw_data)

2

In [98]:
raw_data[0].insert(0, twh_header)
raw_data[1].insert(0, percent_header)

In [99]:
raw_data[0].pop(1)
raw_data[1].pop(1)

'Location Renew. Hydro Wind Solar Bio. Geo. Year'

In [103]:
print(raw_data[0][:5])
print(raw_data[1][:5])

[['Location', 'Renew', 'Hydro', 'Wind', 'Solar', 'Bio', 'Geo', 'Year'], ' World 8,971 4,240 2,310 1,644 689 88 2023', ' China 2,894 1,226 886 584 198 0.2 2023', ' United States 965 239 421 239 47 18 2023', ' Brazil 631 429 96 51 56 0 2023']
[['Location', 'Renew', 'Hydro', 'Wind', 'Solar', 'Bio', 'Geo', 'Year'], ' World 30 14 8 6 2 0.3 2023', ' Albania 100 97 0 3 0 0 2023', ' Bhutan 100 100 0 0 0 0 2023', ' Central African Republic 100 100 0 0 0 0 2023']


In [104]:
# Process rows: split strings and remove commas from numbers
twh_rows = []
percent_rows = []
for row1 in raw_data[0][1:]:
    values1 = row1.replace(',', '').rsplit(maxsplit=7)
    twh_rows.append(values1)

for row2 in raw_data[1][1:]:
    values2 = row2.replace(',', '').rsplit(maxsplit=7)
    percent_rows.append(values2)

In [105]:
twh_df = pd.DataFrame(twh_rows, columns=twh_header)
percent_df = pd.DataFrame(percent_rows, columns=percent_header)

In [106]:
# Convert numeric columns to appropriate types
numeric_cols = ['Renew', 'Hydro', 'Wind', 'Solar', 'Bio', 'Geo', 'Year']
twh_df[numeric_cols] = twh_df[numeric_cols].apply(pd.to_numeric)
percent_df[numeric_cols] = percent_df[numeric_cols].apply(pd.to_numeric)
twh_df['Location'] = twh_df['Location'].str.strip()
percent_df['Location'] = percent_df['Location'].str.strip()

In [107]:
print(len(twh_df), len(percent_df))

213 213


In [96]:
twh_df

,Location,Renew,Hydro,Wind,Solar,Bio,Geo,Year
0,World,8971.0,4240.0,2310.0,1644.0,689.0,88.0,2023
1,China,2894.0,1226.0,886.0,584.0,198.0,0.2,2023
2,United States,965.0,239.0,421.0,239.0,47.0,18.0,2023
3,Brazil,631.0,429.0,96.0,51.0,56.0,0.0,2023
4,Canada,421.0,364.0,39.0,8.0,10.0,0.0,2023
...,...,...,...,...,...,...,...,...
208,Saint Helena Ascension and Tristan da Cunha,0.0,0.0,0.0,0.0,0.0,0.0,2023
209,Saint Pierre and Miquelon,0.0,0.0,0.0,0.0,0.0,0.0,2023
210,Timor-Leste,0.0,0.0,0.0,0.0,0.0,0.0,2023
211,Turks and Caicos Islands,0.0,0.0,0.0,0.0,0.0,0.0,2023


In [108]:
percent_df

,Location,Renew,Hydro,Wind,Solar,Bio,Geo,Year
0,World,30.0,14.0,8.0,6.0,2.0,0.3,2023
1,Albania,100.0,97.0,0.0,3.0,0.0,0.0,2023
2,Bhutan,100.0,100.0,0.0,0.0,0.0,0.0,2023
3,Central African Republic,100.0,100.0,0.0,0.0,0.0,0.0,2023
4,DR Congo,100.0,86.0,0.0,14.0,0.2,0.0,2023
...,...,...,...,...,...,...,...,...
208,Saint Helena Ascension and Tristan da Cunha,0.0,0.0,0.0,0.0,0.0,0.0,2023
209,Saint Pierre and Miquelon,0.0,0.0,0.0,0.0,0.0,0.0,2023
210,Timor-Leste,0.0,0.0,0.0,0.0,0.0,0.0,2023
211,Turks and Caicos Islands,0.0,0.0,0.0,0.0,0.0,0.0,2023


In [97]:
twh_df.to_csv("renewable_twh.csv", index=False)

In [109]:
percent_df.to_csv("renewable_percent.csv", index=False)